In [20]:
import numpy as np
import csv
# from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from model import Network, SimpleNET, ResNet
    
from augmentation import transforms_0
from train import train

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
BATCH_SIZE = 64
trainset = datasets.ImageFolder('/dtu/datasets1/02514/hotdog_nothotdog/train', transform=transforms_0)
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

testset = datasets.ImageFolder('/dtu/datasets1/02514/hotdog_nothotdog/test', transform=transforms_0)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

In [28]:
test_classes = [re.split("/| ", c)[-2] for c, _ in testset.imgs]
classes = set([re.split("/| ", c)[-2] for c, _ in testset.imgs])

### Train the model (to be replaced with loading the model)

In [21]:
MODEL_NAME = 'ResNet'
if MODEL_NAME == 'ResNet':
    from config.ResNet_config import *
else:
    from config.baseline_config import *

In [22]:
if MODEL_NAME == 'ResNet':
    model = eval(MODEL_NAME)(n_features=CONV_SIZE, num_res_blocks=NUM_BLOCKS)
else:
    model = eval(MODEL_NAME)()
    
model.to(device)
loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) # , weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=DECAY_RATE)

In [23]:
out_dict_adam = train(model, train_loader, test_loader, loss, optimizer, NUM_EPOCHS)

  0%|          | 0/20 [00:00<?, ?epoch/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 0.702	 test: 0.679	 Accuracy train: 52.7%	 test: 53.0%


  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 0.650	 test: 0.615	 Accuracy train: 62.9%	 test: 69.0%


...

Loss train: 0.240	 test: 0.580	 Accuracy train: 91.0%	 test: 72.1%


  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 0.182	 test: 0.657	 Accuracy train: 94.2%	 test: 72.6%


  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 0.142	 test: 0.592	 Accuracy train: 96.4%	 test: 73.1%


### Get misclassifications

In [24]:
test_output = None
test_target = None
for data, target in test_loader:
    data = data.to(device)
    with torch.no_grad():
        output = model(data)
        
    if test_output is None:
        test_output = output
        test_target = target
    else:
        test_output = torch.cat((test_output, output))
        test_target = torch.cat((test_target, target))

In [64]:
test_predicted = (test_output > 0.5).type(torch.IntTensor)[:, 0]

In [95]:
acc = dict([(c, 0.) for c in classes])
for c in classes:
    acc[c] = ((test_predicted == test_target) * (np.array(test_classes) == c)).sum().float() / ((np.array(test_classes) == c).sum()).astype(float)

In [96]:
acc

{'food': tensor(0.4377),
 'hotdog': tensor(0.7355),
 'furniture': tensor(0.7634),
 'pets': tensor(0.7721),
 'people': tensor(0.9147),
 'chilidog': tensor(0.7727),
 'frankfurter': tensor(0.7243)}